In [ ]:
%pip install -q -r requirements.txt

In [1]:
# Importando tudo que a análise econometrica precisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pyreadstat
import seaborn as sns
import pandasql as ps

In [2]:
# Importando o dataset do arquivo .sav
df_general, meta1 = pyreadstat.read_sav('BRUNO FRANCISCO SCHADEN Data Req 8.21.24/Deidentified_WPH003a Economy (General Population) (SPSS Version).sav')

df_economists, meta2 = pyreadstat.read_sav('BRUNO FRANCISCO SCHADEN Data Req 8.21.24/Deidentified_WPH003b Economy (Economists) (SPSS Version).sav')

In [7]:
df_general['econ'] = 0
df_economists['econ'] = 1

In [8]:
# Union nas duas tabelas usando as colunas em comum
df = pd.concat([df_general, df_economists], axis=0)

In [9]:
df

,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,q36e1,q36f1,q36g1,q115,seq,pri,sic,title,effort,expdate
0,1.0,1.0,1.0,2.0,1950.0,4.0,1.0,2.0,37.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2.0,2.0,5.0,1967.0,7.0,1.0,2.0,37.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,1.0,2.0,3.0,1970.0,2.0,1.0,2.0,27.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,3.0,1.0,3.0,1942.0,7.0,1.0,2.0,37.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,3.0,1.0,3.0,1956.0,7.0,1.0,1.0,35.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,595.0,2.0,1.0,3.0,1955.0,7.0,1.0,1.0,10.0,2.0,...,3.0,3.0,3.0,2.0,1966.0,0.0,5.0,99.0,,9709.0
595,596.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,1.0,...,NaN,NaN,NaN,NaN,332.0,0.0,0.0,99.0,,9612.0
596,597.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,23.0,3.0,...,NaN,NaN,NaN,NaN,1367.0,0.0,0.0,99.0,,9706.0
597,598.0,2.0,1.0,4.0,1930.0,9.0,1.0,1.0,49.0,4.0,...,3.0,3.0,3.0,2.0,2830.0,0.0,0.0,99.0,,9706.0


In [ ]:
# Selecionando as colunas para análise
df_raw = df[['econ', 'race', 'q910', 'sex', ]]

In [5]:
# lendo a descrição das colunas
meta1.column_labels

[None,
 'PARTY CONSIDER SELF',
 'REGISTERED TO VOTE',
 'LIBERAL,MODERATE,CONSERVATIVE',
 'YEAR OF BIRTH',
 'INCOME',
 'ETHNICITY,RACE',
 'SEX',
 None,
 'REGION',
 None,
 None,
 None,
 None,
 'STATE OF AMERICAN ECONOMY?',
 'MOST IMPORTANT ECONOMIC - 1ST CHOICE?',
 'MOST IMPORTANT ECONOMIC - 2ND CHOICE?',
 '5 YEARS AGO: UNEMPLOYMENT RATE?',
 '5 YEARS AGO: INFLATION RATE?',
 '% OF AMERICANS UNEMPLOYED, LOOKING FOR WORK?',
 'CURRENT ANNUAL RATE OF INFLATION?',
 '% OF PROFIT AMERICAN CORPORATIONS?',
 '5 YEARS AGO: FEDERAL BUDGET DEFICIT?',
 'PAST 5 YEARS: # OF FULL-TIME JOBS?',
 'NEW JOBS: PAY WELL OR LOW PAY?',
 'PAST 20 YEARS: GAP BETWEEN RICH AND POOR?',
 'DIRECTION OF AVERAGE INCOMES OVER LAST 20 YEARS?',
 'DIRECTION OF WAGES OVER THE LAST 20 YEARS?',
 'TWO FULL-TIME WAGE EARNERS VS. ONE?',
 'DIRECTION OF STANDARD OF LIVING DURING NEXT 5 YEARS?',
 'YOUR FAMILY INCOME OVER THE PAST 5 YEARS?',
 'FAMILY INCOME - 1ST CHOICE?',
 'FAMILY INCOME - 2ND CHOICE?',
 'FAMILY INCOME OVER THE NEXT 5 

In [ ]:
df_general.to_excel('BRUNO FRANCISCO SCHADEN Data Req 8.21.24/Deidentified_WPH003a Economy (General Population) (SPSS Version).xlsx')

In [21]:
df_economists = pd.read_spss('BRUNO FRANCISCO SCHADEN Data Req 8.21.24\Deidentified_WPH003b Economy (Economists) (SPSS Version).sav')
df_economists.to_excel('BRUNO FRANCISCO SCHADEN Data Req 8.21.24\Deidentified_WPH003b Economy (Economists) (SPSS Version).xlsx')

In [19]:
df_general.head()

,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,q46,q47,q48,q49,q50,q51,q55,q56,q919,wt1
0,1.0,1.0,1.0,2.0,1950.0,4.0,1.0,2.0,37.0,3.0,...,6.0,NaN,2.0,NaN,NaN,2.0,5.0,6.0,2.0,0.89
1,2.0,2.0,2.0,5.0,1967.0,7.0,1.0,2.0,37.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,7.0,2.0,1.00
2,3.0,1.0,2.0,3.0,1970.0,2.0,1.0,2.0,27.0,3.0,...,NaN,NaN,2.0,NaN,NaN,2.0,3.0,NaN,2.0,1.23
3,4.0,3.0,1.0,3.0,1942.0,7.0,1.0,2.0,37.0,3.0,...,1.0,1.0,2.0,NaN,NaN,2.0,5.0,5.0,2.0,0.88
4,5.0,3.0,1.0,3.0,1956.0,7.0,1.0,1.0,35.0,3.0,...,1.0,1.0,2.0,NaN,NaN,1.0,5.0,5.0,2.0,0.94


In [22]:
df_economists.head()

,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,q44,q45,q919,q115,seq,pri,sic,title,effort,expdate
0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,24.0,SOUTH,...,NaN,NaN,NaN,NaN,1197.0,3.0,0.0,99.0,,9706.0
1,2.0,INDEPENDENT,YES,MODERATE,1937.0,$100000 OR MORE,WHITE,MALE,24.0,SOUTH,...,YES,STAY SAME,NO,LARGER,1028.0,0.0,0.0,0.0,N50301,9704.0
2,3.0,INDEPENDENT,YES,MODERATE,1942.0,$75000-99999,WHITE,MALE,24.0,SOUTH,...,YES,STAY SAME,NO,LARGER,845.0,0.0,0.0,99.0,,9706.0
3,4.0,DEMOCRAT,YES,MODERATE,1951.0,$50000-74999,WHITE,MALE,24.0,SOUTH,...,NO,NaN,NO,LARGER,836.0,0.0,0.0,99.0,,9612.0
4,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,24.0,SOUTH,...,NaN,NaN,NaN,NaN,933.0,0.0,0.0,99.0,,9703.0
